In [1]:
import pandas as pd
import pandas_ta as ta
import datetime

In [2]:
from st2 import apply_pivots

In [3]:
# Load intraday data from CSV with pandas
df = pd.read_csv(r'/Users/suganeshr/Trading/Trading_codes/nifty_200/ABB.csv', parse_dates=True, index_col='Datetime')

# Ensure the index is named 'datetime' and is in the correct format
df.index.name = 'datetime'
df.index = pd.to_datetime(df.index)

# Resample data to get daily OHLC
daily_df = df.resample('D').agg({
    'Open': 'first',
    'High': 'max',
    'Low': 'min',
    'Close': 'last',
    'Volume': 'sum'
}).shift(1).dropna()



In [4]:

# Calculate Pivot Points and other levels using previous day's OHLC
data_with_pivots = apply_pivots(daily_df)


In [5]:
data_with_pivots

,Open,High,Low,Close,Volume,Prev_High,Prev_Low,Prev_Close,PP,TC,BC,R1,R2,R3,S1,S2,S3,%CPR,Date
datetime,,,,,,,,,,,,,,,,,,,
2020-07-04,965.00,988.00,940.00,979.50,320464.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-07-04
2020-07-07,982.00,1017.00,976.20,983.40,397991.0,988.00,940.00,979.50,969.166667,978.583333,954.583333,998.333333,1017.166667,1046.333333,950.333333,921.166667,902.333333,0.024764,2020-07-07
2020-07-08,987.50,988.95,957.25,961.40,298211.0,1017.00,976.20,983.40,992.200000,1004.600000,984.200000,1008.200000,1033.000000,1049.000000,967.400000,951.400000,926.600000,0.020560,2020-07-08
2020-07-09,968.00,971.50,942.00,944.65,194538.0,988.95,957.25,961.40,969.200000,979.075000,963.225000,981.150000,1000.900000,1012.850000,949.450000,937.500000,917.750000,0.016354,2020-07-09
2020-07-10,947.00,955.00,926.05,928.50,124564.0,971.50,942.00,944.65,952.716667,962.108333,947.358333,963.433333,982.216667,992.933333,933.933333,923.216667,904.433333,0.015482,2020-07-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-03,7693.00,7779.95,7547.55,7574.00,195192.0,7948.00,7735.00,7749.00,7810.666667,7879.333333,7772.833333,7886.333333,8023.666667,8099.333333,7673.333333,7597.666667,7460.333333,0.013635,2024-08-03
2024-08-06,7401.15,7597.90,7236.80,7450.00,333387.0,7779.95,7547.55,7574.00,7633.833333,7706.891667,7590.691667,7720.116667,7866.233333,7952.516667,7487.716667,7401.433333,7255.316667,0.015222,2024-08-06
2024-08-07,7446.75,7650.00,7428.00,7460.00,269939.0,7597.90,7236.80,7450.00,7428.233333,7513.066667,7332.516667,7619.666667,7789.333333,7980.766667,7258.566667,7067.133333,6897.466667,0.024306,2024-08-07


In [8]:
df['Date']= df.index.date

In [12]:
def calculate_indicators(df):
    df['rsi'] = ta.rsi(df['Close'], length=14).astype('float64')
    df['mfi'] = ta.mfi(df['High'], df['Low'], df['Close'], df['Volume'], length=14)

    macd = ta.macd(df['Close'], fast=12, slow=26, signal=9)
    df['macd'] = macd['MACD_12_26_9'].astype('float64')
    df['macd_signal'] = macd['MACDs_12_26_9'].astype('float64')

    bb = ta.bbands(df['Close'], length=20, std=2)
    df['bb_upper'] = bb['BBU_20_2.0'].astype('float64')
    df['bb_lower'] = bb['BBL_20_2.0'].astype('float64')

    # Calculate %b (Percent B) for Bollinger Bands
    df['percent_b'] = ((df['Close'] - df['bb_lower']) / (df['bb_upper'] - df['bb_lower']) * 100).astype('float64')

    stoch = ta.stoch(df['High'], df['Low'], df['Close'], k=14, d=3)
    df['stoch_k'] = stoch['STOCHk_14_3_3'].astype('float64')
    df['stoch_d'] = stoch['STOCHd_14_3_3'].astype('float64')

    adx = ta.adx(df['High'], df['Low'], df['Close'], length=14)
    df['adx'] = adx['ADX_14'].astype('float64')
    return  df

# Remove NaN values that may result from indicator calculation


/var/folders/ds/wkxfdzns0q9b361mzl9c0_840000gn/T/ipykernel_78549/3884271132.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[9.96385073e+06 1.09381081e+07 9.50225430e+06 ... 7.91434731e+07
 1.98153594e+08 1.79748375e+08]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df['mfi'] = ta.mfi(df['High'], df['Low'], df['Close'], df['Volume'], length=14)
/var/folders/ds/wkxfdzns0q9b361mzl9c0_840000gn/T/ipykernel_78549/3884271132.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[1.94097189e+07 1.17715059e+07 1.31058958e+07 ... 9.73472115e+07
 1.36744774e+08 1.04697527e+08]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df['mfi'] = ta.mfi(df['High'], df['Low'], df['Close'], df['Volume'], length=14)


In [10]:
merged_df = pd.merge(df, data_with_pivots[['Date', 'PP', 'TC', 'BC', 'R1', 'R2', 'R3', 'S1', 'S2', 'S3', '%CPR']],
                         left_on='Date', right_on='Date', how='left')

In [11]:
merged_df

,Open,High,Low,Close,Volume,rsi,mfi,macd,macd_signal,bb_upper,...,PP,TC,BC,R1,R2,R3,S1,S2,S3,%CPR
0,965.00,968.55,940.00,961.20,4790,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,960.00,962.00,941.00,946.90,20432,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,948.00,948.10,945.10,948.10,12429,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,948.10,954.75,947.00,948.05,10489,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,948.10,950.00,943.75,950.00,13826,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76110,7963.50,7975.40,7958.35,7971.85,9932,44.160656,45.645442,-8.536961,-9.081630,7996.111726,...,7757.483333,7836.241667,7613.916667,8044.616667,8202.133333,8489.266667,7599.966667,7312.833333,7155.316667,0.028659
76111,7971.85,7995.00,7969.00,7994.60,24812,53.591807,55.149180,-6.395121,-8.544328,7998.476927,...,7757.483333,7836.241667,7613.916667,8044.616667,8202.133333,8489.266667,7599.966667,7312.833333,7155.316667,0.028659
76112,7994.30,8000.00,7983.05,7983.40,22500,49.187287,66.051047,-5.537610,-7.942985,7998.933034,...,7757.483333,7836.241667,7613.916667,8044.616667,8202.133333,8489.266667,7599.966667,7312.833333,7155.316667,0.028659
76113,7983.55,7983.55,7950.00,7952.00,17175,39.408475,57.013116,-7.307508,-7.815889,8000.427716,...,7757.483333,7836.241667,7613.916667,8044.616667,8202.133333,8489.266667,7599.966667,7312.833333,7155.316667,0.028659
